In [1]:
from datetime import datetime
from netCDF4 import Dataset as ds
import numpy as np
import dateutil as du 
import parser
import pytz
import dateutil.parser 
import pandas as pd
from pandas import DataFrame
import csv
import glob, os
from os.path import expanduser
from pathlib import Path
import xarray as xr 

In [2]:
### Load .CSV and 
### subset bleaching events (conditional, e.g. exclude NaN)
### Filter only CodeID, site names and country" 
df = pd.read_csv('CBdbV2.csv',encoding='latin-1', low_memory=False)
df = df[['RECORD_ID','COUNTRY','LATITUDE','LONGITUDE','SEVERITY_CODE','DATE','MONTH','YEAR']]
df = pd.DataFrame(df)
#### Convert index into column
#df.reset_index(level=0, inplace=True)
df['ITEM_ID'] = df.index.astype(str).str.zfill(7)   # add '0s' at the beginning of the number and convert as string 
df['DAY'] = df['DATE']   ## create a column named DAY to be used with datetime function
df = df[pd.to_numeric(df['DAY'], errors='coerce').notnull()]  # Drop rows that do not have as.numeric DATE (DAY)
df['full_date'] = df.apply(lambda x: '%s-%s-%s' % (x['YEAR'], int(x['MONTH']), x['DAY']), axis=1) # create a column in datetime format; %s - String (or any object with a string representation, like numbers), %d - Integers %f - Floating point numbers
df['full_date'] = df['full_date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) # use datetime funtion to convert string 'full_date' to datetime  
### Re-order by column starting by 'ID' and renaming LAT and LON
df = df.reindex(columns=['ITEM_ID','RECORD_ID','COUNTRY','LATITUDE','LONGITUDE','SEVERITY_CODE','DAY','MONTH','YEAR','full_date','DATE'])
df.rename(columns={'LATITUDE': 'lat', 'LONGITUDE': 'lon'}, inplace=True) # use "implace = True" to rename the existing DataFrame (rather than creating a copy)
df = df.dropna(axis=0, how='any')  # Drop row with NaN if any
df = df[~df.lat.str.contains("Missing")]  # Drop rows with "string" values and not NaN
df = df[~df.lon.str.contains("Missing")]

### Convert df to dictionary
#sites_short = df.to_dict('index')


In [3]:
# Function to slice the filename with .nc extention, this will be used to open the files in directory
def file_info(ncfilename):
    with ds(ncfilename,'r') as ncdat:
        #string = str(ncfilename)
        #filename = string[-16:] 
        array = xr.open_dataset(ncfilename) 
        return array  
def array_sel(array):
    dsloc = array.sel(lon=lon,lat=lat,time=time,method='nearest') 
    cloud_fraction = dsloc['cloud_fraction']
    return cloud_fraction 

In [4]:
home = Path(expanduser("~"))
full_date = [] 
year_list = [] 
for row in df.itertuples():    # itertuples creates a list based on df rows 
    row.YEAR, row.full_date
    year_list.append(row.YEAR) 
    full_date.append(str(row.full_date)) 
data_path_list = [] 
for year in year_list:
    datapath = home / Path("C:/Users/bmped/Documents/UBC/CCEL/Data and methods/Chapter02/patmos_x_" + str(year) + ".nc") # "E:/PGonzalez/PATMOS_X/PATMOS_X_(daily_values_per_year)/" + str(year) + ".nc"
    data_path_list.append(datapath)
df['path'] = data_path_list
x = []
for row in df.itertuples():
        lat = row.lat,
        lon = row.lon
        time = row.full_date
        path = row.path
        array = file_info(path)
        cf = array_sel(array)
        x.append(cf)
cloud = [item[0].values for item in x] # this will create a list of arrays that contains the "value" of cloud_fraction
df['CF'] = cloud
df.to_csv('site_names.csv')

In [ ]:
cloud = [item[0].values for item in x] # this will create a list of arrays that contains the "value" of cloud_fraction
cloud # the output should be the same lenght as the data frame (df) 


In [ ]:
df['CF'] = cloud

In [ ]:
df.to_csv('site_names.csv')